The following are Tree based models with the hyperparameters for grid search

In [1]:
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import accuracy_score, r2_score
from DataModel import DataModel, TensorDataSet, TensorTabDataSet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

dm = DataModel(encoding_type='label')

train_X, train_y = dm.get_train_data()

test_X, test_y = dm.get_test_data()






First: some data analysis and visualisation:

In [ ]:
#Boxplots of discharge time for each voyage_id (i.e. each vessel) and each parkinng place:
import seaborn as sns

#box plot distribution for discharg time for every voyage id
sns.catplot(data=data[data["discharge_time_minutes"] < 350 ], x=str("voyage_id"), y="discharge_time_minutes", kind= "box",height=15, aspect=1.5)
plt.ylim(0, 350)
plt.xticks(rotation = 90)
plt.xlabel('Vessel Id', fontsize = 35, weight='bold')
plt.yticks(fontsize=30, weight='bold')
plt.ylabel('Discharge time [min]', fontsize= 35, weight='bold')
plt.show()

In [ ]:
#plot discharg time for each voyage id. 
import seaborn as sns
sns.catplot(data=data, x=str("voyage_id"), y="discharge_time_minutes",height=10, aspect=1.5)
plt.ylim(0, 3000)
plt.xticks(rotation = 90)
plt.xlabel('Vessel Id', fontsize = 20, weight='bold')
plt.yticks(fontsize=20, weight='bold')
plt.ylabel('Discharge time [min]', fontsize= 20, weight='bold')
plt.show

In [ ]:
from dython.nominal import associations
dm = DataModel(encoding_type='label')

dd = dm.get_df()
dd.drop('unitype_id', axis=1, inplace=True)
print(dd.shape)
# prepare data matrix for corrolation 
columns_naming = ['Length', 'Gross Weight', 'Place on Board', 'Parking Place',  'Priority', 'Hazardous', 'Reefer', 'Discharge Time [min]', 'Vessel Deck', 'Deck Stowed Order']

# prepare correlation function 
cor_df = dd.corr(method = "pearson").round(2)
print(cor_df.shape, cor_df)

# Prepare plot 
# Step 1: Initiating a fig and axis object
fig, ax = plt.subplots(figsize = (12, 10))
# Step 2: Create a plot
cax = ax.imshow(cor_df.values, interpolation='nearest', cmap = 'Reds', vmin = -1, vmax = 1)
# Step 3: Set axis tick labels 
ax.set_xticks(ticks = range(len(dd.columns)),
              labels = columns_naming )
ax.set_yticks(ticks = range(len(dd.columns)),
              labels = columns_naming )
# Step 4: Resize the tick parameters
ax.tick_params(axis = "x", labelsize = 12, labelrotation = 90)
ax.tick_params(axis = "y", labelsize = 12, labelrotation = 0)
# Step 5: Adding a color bar
fig.colorbar(cax).ax.tick_params(labelsize = 12)
# Step 6: Add annotation
for (x, y), t in np.ndenumerate(cor_df):
    ax.annotate("{:.2f}".format(t),
                 xy = (x, y),
                 va = "center", # vertical position
                 ha = "center") # horizontal position

In [ ]:
#PCA Analysis
import plotly.express as px
pca = PCA(4)
pca.fit(X)
print(pca.explained_variance_ratio_)
# X = pca.transform(X)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

Second: From here GBDT modeling start. Grid search of the following models is iterated for different parameters. 

In [4]:
# Grid search of hyperparameters for (xgbooster model)
# Define estimator 
xgb_r = xg.XGBRegressor( seed = 123)
# Define Grid 
param_grid = { 
    'objective': ['reg:squarederror'],
    'n_estimators': [ 2000, 1000],
    'learning_rate': [0.01],
    'max_depth' : [100, 10], 
    'colsample_bylevel': [0.33, 1] ,
    'subsample' : [0.5,1,0.33],
    'colsample_bytree' : [ 0.33, 1] 
}

# Instantiation
xgb_r_grid_src = GridSearchCV(estimator = xgb_r , param_grid=param_grid)

# Fitting Model 
xgb_r_grid_src.fit(train_X, train_y)

# Best parameters 
print('best parameters',xgb_r_grid_src.best_params_)
# Best score
print('best score', xgb_r_grid_src.best_score_)

best parameters {'colsample_bylevel': 0.33, 'colsample_bytree': 1, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 1000, 'objective': 'reg:squarederror', 'subsample': 1}
best score 0.4407750654600271


In [ ]:
# Grid search of hyperparameters for (xgbooster model)
# Define estimator 
xgb_r = xg.XGBRegressor( seed = 123)
# Define Grid 
param_grid = { 
    'objective': ['reg:squarederror','reg:squaredlogerror','reg:pseudohubererror'],
    'n_estimators': [1,10,100,1000,10000],
    'learning_rate': [0.01, 0.1, 0.25, 0.5, 0.75, 1],
    'max_depth' : [1,10,100,1000,10000,30000], 
    'colsample_bylevel': [0.1,0.25,0.5,0.75,1] ,
    'subsample' : [0.1,0.25,0.5,0.75,1],
    'colsample_bytree' : [0.1,0.25,0.5,0.75,1] 
}


# Instantiation
xgb_r_grid_src = GridSearchCV(estimator = xgb_r , param_grid=param_grid)

# Fitting Model 
xgb_r_grid_src.fit(train_X, train_y)

# Best parameters 
print('best parameters',xgb_r_grid_src.best_params_)
# Best score
print('best score', xgb_r_grid_src.best_score_)

In [2]:
# Grid search of hyperparameters for (Random forest) regressor model 
# Define estimator 

randomforest_r = RandomForestRegressor( random_state=123)

#define parameter to be searched 
param_grid = {
    'criterion' : ['squared_error'],
    'n_estimators': [10,100,1000], 
    'max_depth' : [1,500,3000],
    'max_features' : [ 0.33, 0.66]
    }

# Instantiation
rndm_frst_grid_src = GridSearchCV(estimator = randomforest_r , param_grid=param_grid)

# fitting model
rndm_frst_grid_src.fit(train_X, train_y)

#best parameters 
print('best parameters', rndm_frst_grid_src.best_params_)
#best score
print('best score', rndm_frst_grid_src.best_score_)
#test score
accuracy = rndm_frst_grid_src.score(test_X, test_y)
print('Test score:', accuracy)


best parameters {'criterion': 'squared_error', 'max_depth': 500, 'max_features': 0.33, 'n_estimators': 1000}
best score 0.4289083355197888
Test score: 0.4435324522943379


In [ ]:
# Grid search of hyperparameters for (graident boosting) regressor model
# Define estimator 
gb_r = GradientBoostingRegressor(random_state=123)

#define parameter to be searched 
param_grid = {
    'criterion' : ['squared_error'] ,
    'n_estimators': [100],
    'max_depth' : [100],
    'max_features' : [0.25]
    }

# Instantiation
gb_r_grid_src = GridSearchCV(estimator = gb_r , param_grid=param_grid)

# fitting model
gb_r_grid_src.fit(train_X, train_y)

print('best parameters',gb_r_grid_src.best_params_)
#best score
print('best score',gb_r_grid_src.best_score_)

In [ ]:
# Grid search of hyperparameters for (Ada booster) model 
# Define estimator 
ada_r = AdaBoostRegressor(random_state=123)

#define parameter to be searched 
param_grid = {
    'n_estimators' : [1, 10, 100, 500, 1000,10000], 
    'learning_rate' : [0.1, 0.5, 1, 2, 5, 10, 100, 1000]
    }

# Instantiation
ada_r_grid_src = GridSearchCV(estimator = ada_r , param_grid=param_grid)

# fitting model
ada_r_grid_src.fit(train_X, train_y)

print('best parameters',ada_r_grid_src.best_params_)
#best score
print('best score',ada_r_grid_src.best_score_)

